# Uma visão geral dos atletas das Olimpiadas (COLOCAR FOTOS)

### Integrantes:
- Caio Lucas da Silva Chacon 20200025769
- Luiz Fernando Costa dos Santos 20200025446
- Moisés Filipe Coutinho Correia Santos 20200025689

## Introdução e objetivos

Após o atraso das Olímpiadas de Tokyo em 2020 devido a pandemia do novo Coronavírus SARSCOVID-19, finalmente foi anunciada a nova data para a realização do evento: 24/07/2021. Com menos de um mês para o início da maior e mais consagrada competição esportiva do mundo, iremos desbravar o histórico do torneio e fazer uma análise descritiva dos seus dados.

Para tanto, os conceitos de Data-driven journalism e os princípios da visualização de dados serão levados em consideração, a fim de criar uma história digna de capa de jornal e agradar os leitores.

### Objetivo geral:
- Apresentar os dados dos atletas olímpicos de forma simplificada.

### Objetivos específicos:
- Descobrir como o peso e a altura dos atletas influenciam na quantidade de medalhas obtidas;
- Quantificar as medalhas por país ao longo do tempo;
- Entender a participação das mulheres nas olimpiadas;
- Encontrar os maiores medalhistas da história;

#### Bibliotecas utilizadas

Para a análise, as seguintes bibliotecas foram utilizadas:

Você pode encontrar a versão delas [nessa página](google.com)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import plotly.graph_objects as go
import requests

from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from IPython.core.display import display, HTML

sns.set_palette("twilight")
warnings.filterwarnings("ignore")

#### Dados

Os dados foram obtidos no [Kaggle](https://www.kaggle.com/heesoo37/120-years-of-olympic-history-athletes-and-results) e serviram como base para todo o projeto. O arquivo athlete_events.csv contém 271116 linhas e 15 colunas, abrangendo os jogos de Athenas em 1896 até os jogos no Rio em 2016. Cada linha corresponde a um atleta individual competinto em um evento olimpico. As colunas são:

- ID (inteiro): Número único para cada atleta;
- Name (string): Nome do atlet;
- Sex (string): Sexo do atleta (Masculino ou Feminino);
- Age (inteiro): Idade do atleta;
- Height (float): Altura do atleta (em centimetros);
- Weight (float): Peso do atleta (em kilogramas);
- Team (string): Nome do time;
- NOC (string): Código de 3 letras do Comitê Olímpico Nacional para um país;
- Games (string): Ano e estação em que a edição da olimpíada acontece;
- Year (inteiro): Ano em que a edição da olimpíada acontece;
- Season (string): Estação em que a olimpíada acontece (verão ou inverno);
- City (string): Cidade sede da edição das olimpíadas;
- Sport (string): Modalidade esportiva que o atleta participou;
- Event (string): Descrição da categoria evento esportivo;
- Medal (string): Informa qual medalha o atleta ganhou (ouro, prata, bronze ou nenhuma[nan]). 

In [ ]:
PATH = "athlete_events.csv" 

df = pd.read_csv(PATH)  
df.head()

Aqui é mensurado a centralidade e dispersão da altura, peso e idade dos atletas

In [ ]:
#chosing the columns to be used
physiological_cols = ["Age", "Height", "Weight"]

df[physiological_cols].describe().iloc[1:,]

Percebe-se que boa parte dos atletas estão na faixa dos 21 e 28 anos, a média de altura é de 175 cm, que é basicamente idêntica à mediana. Assim como o peso, que fica na faixa dos 70 kg.

# TO DO: Deixar a exploração inicial mais completa. Fazer gráficos basicões mesmo das colunas principais. Mostra médias, medianas, tendências. Não agrega nada, coloca apenas o básico.

#### Pré-processamento

O pré-processamento foi feito para resolver o problema com valores ausentes e permitir que novas analises sejam realizadas mais abaixo. Todo o pré-processamento está aqui, porém ele foi realizado em momentos diferentes, adaptados para a necessidade do momento.

In [ ]:
# Procurando por valores ausentes
df.info()

Para transformar os valores faltantes serão usados os valores da média de cada esporte em que o atleta participa. Essa é uma boa escolha porque preserva as principais características de cada esporte. Por exemplo: atletas de basquete são geralmente mais altos que a média dos outros competidores.

In [ ]:
# Cria o dict {sport: mean_age_by_sport}
mean_age_by_sport = df.groupby("Sport").mean().reset_index().drop(columns = ["Height","Year","ID","Weight"])
mean_age_dict = dict(zip(mean_age_by_sport["Sport"],mean_age_by_sport["Age"]))

# Transforma valores nulos pela média de idade do esporte
null_ages = df[pd.isnull(df["Age"])]
null_ages["Age"] = null_ages["Sport"].map(mean_age_dict)
df["Age"].fillna(null_ages["Age"],inplace=True)
# 9474 valores transformados

mean_height_by_sport = df.groupby("Sport").mean().reset_index().drop(columns = ["Age","Year","ID","Weight"])
mean_height_dict = dict(zip(mean_height_by_sport["Sport"],mean_height_by_sport["Height"]))

# Transforma valores nulos pela média de altura do esporte
null_heights = df[pd.isnull(df["Height"])]
null_heights["Height"] = null_heights["Sport"].map(mean_height_dict)
df["Height"].fillna(null_heights["Height"],inplace=True)
# 60072 valores transformados

mean_weight_by_sport = df.groupby("Sport").mean().reset_index().drop(columns = ["Age","Year","ID","Height"])
mean_weight_dict = dict(zip(mean_weight_by_sport["Sport"],mean_weight_by_sport["Weight"]))

# Transforma valores nulos pela média de peso do esporte
null_weights = df[pd.isnull(df["Weight"])]
null_weights["Weight"] = null_weights["Sport"].map(mean_weight_dict)
df["Weight"].fillna(null_weights["Weight"],inplace=True)
df.info() # 62658 values transformed

In [ ]:
# Transforma os valores nulos de medalha por "No medal"
df["Medal"].fillna("No medal",inplace=True)

In [ ]:
# Exclui os valores nulos remanescentes, uma vez que não existem mais boas abordagens 
# para transformá-los
df.dropna(inplace=True)
df.info()

In [ ]:
# Transforma idade, altura e peso em inteiros
df = df.astype({"Age":int,"Height":int,"Weight":int})
df.head()

Em um momento, o software Flourish foi necessário para criar um _"racebar chart"_ , e para isso foi necessário um outro pré-processamento. Abaixo, segue o que foi realizado nele de maneira comentada, para que não aja conflito com os demais códigos:

In [ ]:
# Aqui é escolhido apenas as Olimpíadas de verão e transforma as medalhas em valor booleano
# df = df[df["Season"] == "Summer"]
# df["Medal"] = (df["Medal"].isna()==False)

In [ ]:
# Função criada para que seja possível passar as informações dos dados para o Flourish
def to_flourish(data, var):
    series_VY = data.groupby([var,"Year"]).sum()["Medal"]

    YEARS_QT = data["Year"].unique().size
    VAR_QT = data[var].unique().size

    YEARS = sorted(data["Year"].unique())
    VAR = sorted(data[var].unique())

    array_zeros = np.zeros(shape=(YEARS_QT, VAR_QT))
    data_zeros = pd.DataFrame(data=array_zeros, index=YEARS, columns=VAR)

    for i in VAR:
        data_zeros[i] = data_zeros[i] + series_VY[i].cumsum()
        
    data_final = data_zeros.fillna(method="ffill")
    
    return data_final.T.fillna(0)

In [ ]:
# Primeiro dataset para o flourish, que contém os dados das medalhas divididos pelo NOC
# df_top15 = to_flourish(df, "NOC")



In [ ]:
# O que faltava era preencher o dataframe com as bandeiras de cada país

# PATH_2 = "./data/noc_regions.csv"
# noc_reg = pd.read_csv(PATH_2)

# Função para pegar a URL do NOC dado
def get_link_flag(NOC):
    countr = noc_reg[noc_reg["NOC"] == NOC]["region"].values[0] # The name of the country 
    url = f"https://pt.wikipedia.org/wiki/Ficheiro:Flag_of_{countr}.svg" # the url of the wikipedia flags website
    html = requests.get(url).text 
    soup = BeautifulSoup(html, "html.parser")

    CLASS_ = "fullImageLink" # The class with the image url
    string = str(soup.find_all("div", {"class":CLASS_})[0]) # All the class in str type
    
    # Searching for the scr and .png to get the url 
    start = string.find("src=") + 7 
    end = string.find(".png") + 4
    
    return string[start:end]


# pegar todas as URL's para cada NOC
# not_found = []
# found = []

# for i in noc_reg["NOC"].values:
#     try:
#         found.append("https://"+get_link_flag(i))
#     except IndexError:
#         not_found.append(i)
#         found.append(i)
#         continue
    
# Criando uma nova coluna para as URL's dadas
# df_top15["FLAGS"] = found 
# df_top15.head()

In [ ]:
# Cuidando dos not_found manualmente

# URLS = ["https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/Flag_of_Bolivia_%28state%29.svg/800px-Flag_of_Bolivia_%28state%29.svg.png",
#         "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Flag_of_the_Democratic_Republic_of_the_Congo.svg/800px-Flag_of_the_Democratic_Republic_of_the_Congo.svg.png",
#         "https://upload.wikimedia.org/wikipedia/commons/0/08/Flag_of_the_Individual_Olympic_Athletes_%282000_Summer_Olympics%29.svg",
#         "https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Flag_of_the_United_States_Virgin_Islands.svg/1275px-Flag_of_the_United_States_Virgin_Islands.svg.png",
#         "https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Flag_of_the_British_Virgin_Islands.svg/1200px-Flag_of_the_British_Virgin_Islands.svg.png",
#         "https://compote.slate.com/images/57d479a0-63d2-4462-97b0-1275e4efefe1.jpg",
#         "https://upload.wikimedia.org/wikipedia/commons/f/fe/Flag_of_Saint_Kitts_and_Nevis.svg",
#         "https://www.countryflags.com/wp-content/uploads/trinidad-and-tobago-flag-png-large.png",
#         "https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Flag_of_Tuvalu.svg/1200px-Flag_of_Tuvalu.svg.png",
#         "https://upload.wikimedia.org/wikipedia/commons/2/2e/Unknown_flag_-_European_version.png",
#         "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Flag_of_Saint_Vincent_and_the_Grenadines.svg/1200px-Flag_of_Saint_Vincent_and_the_Grenadines.svg.png",
#         "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Flag_of_the_West_Indies_Federation_%281958%E2%80%931962%29.svg/1200px-Flag_of_the_West_Indies_Federation_%281958%E2%80%931962%29.svg.png"]

# for i,url in enumerate(URLS):
#     df_top15.loc[not_found[i],"FLAGS"] = url

# # This 2 changes is because:
# #                           the NOC of URS was "Russia", but should be "URSS";
# #                           GDR was "Germany", but should be "East Germany".

# df_top15.loc["GDR","FLAGS"] = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Flag_of_East_Germany.svg/1280px-Flag_of_East_Germany.svg.png"
# df_top15.loc["URS","FLAGS"] = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Flag_of_the_Soviet_Union.svg/1200px-Flag_of_the_Soviet_Union.svg.png"

In [ ]:
# exporting the csv for flourish
# PATH_3 = "./data/df_for_flourish_top15.csv"

# df_top15.to_csv(PATH_3)

In [ ]:
# Pegando os dados do Brasil e Estados Unidos para criar novas visualizações no Flourish
# br = df[df["NOC"] == "BRA"]
# us = df[df["NOC"] == "USA"]

# br_final = to_flourish(br, "Sport")
# us_final = to_flourish(us, "Sport")

# PATH_4 = "./data/df_for_flourish_BR.csv"

# br_final.to_csv(PATH_4)

# PATH_5 = "./data/df_for_flourish_US.csv"

# us_final.to_csv(PATH_5)

Racebar...

In [ ]:
flourish_url = "<iframe src='https://flo.uri.sh/visualisation/6409642/embed' title='Interactive or visual content' class='flourish-embed-iframe' frameborder='0' scrolling='no' style='width:100%;height:600px;' sandbox='allow-same-origin allow-forms allow-scripts allow-downloads allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/6409642/?utm_source=embed&utm_campaign=visualisation/6409642' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>"
#geralzao
display(HTML(flourish_url))

In [ ]:
flourish_url_br = "<iframe src='https://flo.uri.sh/visualisation/6500927/embed' title='Interactive or visual content' class='flourish-embed-iframe' frameborder='0' scrolling='no' style='width:100%;height:600px;' sandbox='allow-same-origin allow-forms allow-scripts allow-downloads allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/6500927/?utm_source=embed&utm_campaign=visualisation/6500927' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>"
#apenas br
display(HTML(flourish_url_br))

In [ ]:
flourish_url_us = "<iframe src='https://flo.uri.sh/visualisation/6502031/embed' title='Interactive or visual content' class='flourish-embed-iframe' frameborder='0' scrolling='no' style='width:100%;height:600px;' sandbox='allow-same-origin allow-forms allow-scripts allow-downloads allow-popups allow-popups-to-escape-sandbox allow-top-navigation-by-user-activation'></iframe><div style='width:100%!;margin-top:4px!important;text-align:right!important;'><a class='flourish-credit' href='https://public.flourish.studio/visualisation/6502031/?utm_source=embed&utm_campaign=visualisation/6502031' target='_top' style='text-decoration:none!important'><img alt='Made with Flourish' src='https://public.flourish.studio/resources/made_with_flourish.svg' style='width:105px!important;height:16px!important;border:none!important;margin:0!important;'> </a></div>"
#apenas us
display(HTML(flourish_url_us))